In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import io


In [7]:

with open('CUB_200_2011\images.txt', 'r') as file:
    images = file.read()

df = pd.read_csv(io.StringIO(images), sep=r"\s+", usecols=[1], names=["full_path"])
df["full_path"] = df["full_path"].apply(lambda x: "CUB_200_2011/images/" + x)

In [8]:
data = np.loadtxt('CUB_200_2011/attributes/class_attribute_labels_continuous.txt')
print(data.shape)

(200, 312)


In [11]:
img_att_data = np.loadtxt('CUB_200_2011/attributes/image_attribute_labels_clean.txt')
print(img_att_data.shape)   

(3677856, 5) 3677856


In [13]:
target_lables = np.zeros((11788, 312), dtype=int)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:


for i in range(0, 11788):
    for j in range (0, 312):
        img_att_data[i*312 + j][] = 


In [8]:
df['idx'] = pd.read_csv(io.StringIO(images), sep=r"\s+", usecols=[1], names=["idx"])
df['idx']  = df['idx'].apply(lambda x: int(x.split('.')[0]))
df['target'] = df['idx'].apply(lambda x : data[x-1])
df.drop(columns = ['idx'], inplace = True)
df.head()

,full_path,target
0,CUB_200_2011/images/001.Black_footed_Albatross...,"[0.0, 2.9197080292, 1.4598540146, 0.0, 59.8540..."
1,CUB_200_2011/images/001.Black_footed_Albatross...,"[0.0, 2.9197080292, 1.4598540146, 0.0, 59.8540..."
2,CUB_200_2011/images/001.Black_footed_Albatross...,"[0.0, 2.9197080292, 1.4598540146, 0.0, 59.8540..."
3,CUB_200_2011/images/001.Black_footed_Albatross...,"[0.0, 2.9197080292, 1.4598540146, 0.0, 59.8540..."
4,CUB_200_2011/images/001.Black_footed_Albatross...,"[0.0, 2.9197080292, 1.4598540146, 0.0, 59.8540..."


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        # Input size: (3, 224, 224)
        # After conv1 + pool1 → (32, 112, 112)
        # After conv2 + pool2 → (64, 56, 56)
        self.flattened_size = 64 * 56 * 56

        self.fc1 = nn.Linear(self.flattened_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 312)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
class BirdsAttributesDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['full_path']
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        target = self.df.iloc[idx]['target']
        target = torch.tensor(target, dtype=torch.float32)
        return image, target

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ---- Split the Dataset ----
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# ---- Create Datasets and Dataloaders ----
train_dataset = BirdsAttributesDataset(train_df)
val_dataset = BirdsAttributesDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [13]:
epochs = 10

for epoch in range(epochs):
    print(f"\n[Epoch {epoch+1}] Starting training...")
    model.train()
    running_loss = 0.0

    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
            print(f"  - Training batch {batch_idx + 1}/{len(train_loader)} completed")

    print(f"[Epoch {epoch+1}] Training complete. Starting validation...")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_idx, (imgs, labels) in enumerate(val_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            if (val_batch_idx + 1) % 10 == 0 or (val_batch_idx + 1) == len(val_loader):
                print(f"  - Validation batch {val_batch_idx + 1}/{len(val_loader)} completed")

    print(f"[Epoch {epoch+1}] Train Loss: {running_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")



[Epoch 1] Starting training...


c:\ML_and_DL\birds_species_classifier\env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  - Training batch 10/2358 completed


KeyboardInterrupt: 

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# # ... (your existing code for model definition, optimizer, criterion, etc.) ...

# # Define a path in your Google Drive to save checkpoints
# # The "Transport endpoint is not connected" error typically occurs when the
# # connection to the mounted Google Drive is lost. Re-mounting the drive
# # and ensuring the path is correct usually resolves this issue.
# CHECKPOINT_DIR = "/content/drive/My Drive/Your_Model_Checkpoints/" # Customize this path
# os.makedirs(CHECKPOINT_DIR, exist_ok=True) # Create the directory if it doesn't exist

# # Initialize start_epoch and best_val_loss (or your preferred metric)
# start_epoch = 0
# best_val_loss = float('inf') # Or initialize with a high value if you're tracking accuracy and want to save the best model
# epochs = 10

# # Check if a checkpoint exists and load it to resume training
# checkpoint_path = os.path.join(CHECKPOINT_DIR, "last_checkpoint.pth") # Or choose a more descriptive name
# if os.path.exists(checkpoint_path):
#     print(f"Loading checkpoint from {checkpoint_path}")
#     checkpoint = torch.load(checkpoint_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
#     best_val_loss = checkpoint.get('best_val_loss', float('inf')) # Handle cases where best_val_loss might not be in older checkpoints
#     print(f"Resuming training from epoch {start_epoch} with previous best validation loss: {best_val_loss:.4f}")
# else:
#     print("No checkpoint found. Starting training from scratch.")

# # Your training loop
# for epoch in range(start_epoch, epochs): # Start from 'start_epoch'
#     print(f"\n[Epoch {epoch+1}] Starting training...")
#     model.train()
#     running_loss = 0.0

#     for batch_idx, (imgs, labels) in enumerate(train_loader):
#         imgs, labels = imgs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(imgs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()

#         if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
#             print(f"  - Training batch {batch_idx + 1}/{len(train_loader)} completed")

#     train_loss = running_loss / len(train_loader)
#     print(f"[Epoch {epoch+1}] Training complete. Starting validation...")

#     # Validation
#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for val_batch_idx, (imgs, labels) in enumerate(val_loader):
#             imgs, labels = imgs.to(device), labels.to(device)
#             outputs = model(imgs)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()

#             if (val_batch_idx + 1) % 10 == 0 or (val_batch_idx + 1) == len(val_loader):
#                 print(f"  - Validation batch {val_batch_idx + 1}/{len(val_loader)} completed")

#     current_val_loss = val_loss / len(val_loader)
#     print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f} | Val Loss: {current_val_loss:.4f}")

#     # Save checkpoint
#     checkpoint = {
#         'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'train_loss': train_loss,
#         'val_loss': current_val_loss,
#         'best_val_loss': best_val_loss # Store the best validation loss seen so far
#     }
#     torch.save(checkpoint, checkpoint_path)
#     print(f"Checkpoint saved to {checkpoint_path}")

#     # Optionally, save the best model separately
#     if current_val_loss < best_val_loss:
#         best_val_loss = current_val_loss
#         best_model_path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
#         torch.save(model.state_dict(), best_model_path) # Save only model weights for best model
#         print(f"New best model saved to {best_model_path} with validation loss: {best_val_loss:.4f}")

# print("\nTraining complete!")

ValueError: mount failed